In [1]:
spark

StatementMeta(3c67b279-1d53-4b7a-b0d9-41cb8b4b6723, 56, 6, Finished, Available)

In [2]:
blob_account_name = "marckvnonprodblob"
blob_container_name = "bigdata"
# read only
blob_sas_token = "?sv=2021-10-04&st=2023-10-04T01%3A42%3A59Z&se=2024-01-02T02%3A42%3A00Z&sr=c&sp=rlf&sig=w3CH9MbCOpwO7DtHlrahc7AlRPxSZZb8MOgS6TaXLzI%3D"

wasbs_base_url = (
    f"wasbs://{blob_container_name}@{blob_account_name}.blob.core.windows.net/"
)
spark.conf.set(
    f"fs.azure.sas.{blob_container_name}.{blob_account_name}.blob.core.windows.net",
    blob_sas_token,
)

StatementMeta(3c67b279-1d53-4b7a-b0d9-41cb8b4b6723, 56, 7, Finished, Available)

In [3]:
submissions_path = "reddit-parquet/submissions/"
submissions_df = spark.read.parquet(f"{wasbs_base_url}{submissions_path}")

StatementMeta(3c67b279-1d53-4b7a-b0d9-41cb8b4b6723, 56, 8, Finished, Available)

In [4]:
from pyspark.sql.functions import col, asc,desc

submissions_filtered = submissions_df.filter(col("subreddit").isin(["Minecraft", "Eldenring","Genshin_Impact","pokemongo","RocketLeagueExchange","VALORANT","DestinyTheGame","CallOfDutyMobile","FIFA","pokemon"]))

StatementMeta(3c67b279-1d53-4b7a-b0d9-41cb8b4b6723, 56, 9, Finished, Available)

In [5]:
df_text = submissions_filtered.select("subreddit", "title", "selftext").cache()

StatementMeta(3c67b279-1d53-4b7a-b0d9-41cb8b4b6723, 56, 10, Finished, Available)

In [6]:
from pyspark.sql.functions import concat_ws
df = df_text.withColumn("text", concat_ws(" ", df_text.title, df_text.selftext))

StatementMeta(3c67b279-1d53-4b7a-b0d9-41cb8b4b6723, 56, 11, Finished, Available)

In [7]:
df.show(5)

StatementMeta(3c67b279-1d53-4b7a-b0d9-41cb8b4b6723, 56, 12, Finished, Available)

+---------+--------------------+--------------------+--------------------+
|subreddit|               title|            selftext|                text|
+---------+--------------------+--------------------+--------------------+
|     FIFA|if you every get ...|                    |if you every get ...|
|Minecraft|     Texting a house|So I’m building a...|Texting a house S...|
| VALORANT|       Killjoy Mains|Got bored and so ...|Killjoy Mains Got...|
|Minecraft|Trident skeleton ...|                    |Trident skeleton ...|
|  pokemon|the PokemonTogeth...|So several days a...|the PokemonTogeth...|
+---------+--------------------+--------------------+--------------------+
only showing top 5 rows



In [8]:
from pyspark.sql.functions import lower
df = df.withColumn("text_lower", lower(df.text))

StatementMeta(3c67b279-1d53-4b7a-b0d9-41cb8b4b6723, 56, 13, Finished, Available)

In [9]:
df.show(5)

StatementMeta(3c67b279-1d53-4b7a-b0d9-41cb8b4b6723, 56, 14, Finished, Available)

+---------+--------------------+--------------------+--------------------+--------------------+
|subreddit|               title|            selftext|                text|          text_lower|
+---------+--------------------+--------------------+--------------------+--------------------+
|     FIFA|if you every get ...|                    |if you every get ...|if you every get ...|
|Minecraft|     Texting a house|So I’m building a...|Texting a house S...|texting a house s...|
| VALORANT|       Killjoy Mains|Got bored and so ...|Killjoy Mains Got...|killjoy mains got...|
|Minecraft|Trident skeleton ...|                    |Trident skeleton ...|trident skeleton ...|
|  pokemon|the PokemonTogeth...|So several days a...|the PokemonTogeth...|the pokemontogeth...|
+---------+--------------------+--------------------+--------------------+--------------------+
only showing top 5 rows



# assign keywords labels

In [10]:
keyword_lst = [
    ["bug","error","issue","glitch","crash","freeze","lag","problems","problem","corruption"],
    ["strategies","strategy","tip","trick","tactic","review","playthrough","speedrun","trophy","speedrun","achievements","quests","mission"],
    ["story","storyline","plot","narrative","characters","dialogue","twist","protagonist","antagonist","villian","backstory","emotion","choice","suspense"],
    ["lore","world","mythology","culture","retcon","documents","race","racial","cultural","environment","environmental","artifacts","cosmology"],
    ["trade","buy","sell","transaction","market","scam","valuation","negotiation","currency","credit","auction","rarity","tax","fee","traders","buyers","sellers"]
]

StatementMeta(3c67b279-1d53-4b7a-b0d9-41cb8b4b6723, 56, 15, Finished, Available)

In [17]:
from pyspark.sql.functions import when
df_label = df.withColumn("label",
                   when(col("text_lower").rlike("|".join(keyword_lst[0])), "bug report")
                   .when(col("text_lower").rlike("|".join(keyword_lst[1])), "gameplay discussion")
                   .when(col("text_lower").rlike("|".join(keyword_lst[2])), "story related discussion")
                   .when(col("text_lower").rlike("|".join(keyword_lst[3])), "lore discussion")
                   .when(col("text_lower").rlike("|".join(keyword_lst[4])), "game items trading")
                   .otherwise("other"))
df_label.show(5)

StatementMeta(3c67b279-1d53-4b7a-b0d9-41cb8b4b6723, 56, 22, Finished, Available)

+---------+--------------------+--------------------+--------------------+--------------------+-------------------+
|subreddit|               title|            selftext|                text|          text_lower|              label|
+---------+--------------------+--------------------+--------------------+--------------------+-------------------+
|     FIFA|if you every get ...|                    |if you every get ...|if you every get ...|              other|
|Minecraft|     Texting a house|So I’m building a...|Texting a house S...|texting a house s...|              other|
| VALORANT|       Killjoy Mains|Got bored and so ...|Killjoy Mains Got...|killjoy mains got...| game items trading|
|Minecraft|Trident skeleton ...|                    |Trident skeleton ...|trident skeleton ...|         bug report|
|  pokemon|the PokemonTogeth...|So several days a...|the PokemonTogeth...|the pokemontogeth...|gameplay discussion|
+---------+--------------------+--------------------+-------------------

# check the distribution of each labels

In [18]:
df_label_cnt = df_label.groupBy("label").count().orderBy(col("count"), ascending=False).collect()

StatementMeta(3c67b279-1d53-4b7a-b0d9-41cb8b4b6723, 56, 23, Submitted, Running)

In [21]:
df_label_cnt_tot = spark.createDataFrame(df_label_cnt).toPandas()
df_label_cnt_tot

StatementMeta(3c67b279-1d53-4b7a-b0d9-41cb8b4b6723, 56, 26, Finished, Available)

,label,count
0,other,3925294
1,bug report,358867
2,game items trading,330996
3,gameplay discussion,240189
4,lore discussion,150275
5,story related discussion,86385


In [22]:
df_label_cnt_game = df_label.groupBy("subreddit","label").count().orderBy(col("count"), ascending=False).collect()

StatementMeta(3c67b279-1d53-4b7a-b0d9-41cb8b4b6723, 56, 27, Finished, Available)

In [23]:
df_label_cnt_sub = spark.createDataFrame(df_label_cnt_game).toPandas()
df_label_cnt_sub

StatementMeta(3c67b279-1d53-4b7a-b0d9-41cb8b4b6723, 56, 28, Finished, Available)

,subreddit,label,count
0,Minecraft,other,671646
1,Eldenring,other,634060
2,Genshin_Impact,other,555686
3,pokemongo,other,373640
4,VALORANT,other,317794
5,CallOfDutyMobile,other,312899
6,FIFA,other,297790
7,pokemon,other,291050
8,DestinyTheGame,other,253196
9,RocketLeagueExchange,other,217533


In [24]:
import os
CSV_DIR = os.path.join("Users/ky285/fall-2023-reddit-project-team-11/data", "csv")
df_label_cnt_tot.to_csv(f"{CSV_DIR}/total_label_count.csv", index=False)
df_label_cnt_sub.to_csv(f"{CSV_DIR}/game_label_count.csv", index=False)

StatementMeta(3c67b279-1d53-4b7a-b0d9-41cb8b4b6723, 56, 29, Finished, Available)

# train test validation split

In [25]:
df_ml = df_label.filter((col("label") == "bug report") | (col("label") == "bug report") | (col("label") == "gameplay discussion") | (col("label") == "story related discussion") | (col("label") == "lore discussion") | (col("label") == "game items trading"))

StatementMeta(3c67b279-1d53-4b7a-b0d9-41cb8b4b6723, 56, 30, Finished, Available)

In [26]:
train_data, test_data, val_data = df_ml.randomSplit([0.8, 0.18, 0.02], 24)

StatementMeta(3c67b279-1d53-4b7a-b0d9-41cb8b4b6723, 56, 31, Finished, Available)

In [27]:
print("Number of training records: " + str(train_data.count()))
print("Number of testing records : " + str(test_data.count()))
print("Number of validation records : " + str(val_data.count()))
train_data.cache()

StatementMeta(3c67b279-1d53-4b7a-b0d9-41cb8b4b6723, 56, 32, Finished, Available)

Number of training records: 932198
Number of testing records : 210806
Number of validation records : 23708


DataFrame[subreddit: string, title: string, selftext: string, text: string, text_lower: string, label: string]